In [18]:
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

In [19]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 32


train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=40,
    height_shift_range=0.2,
    width_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_path = 'dataset/'

train_dataset = train_datagen.flow_from_directory(train_path, 
                                               target_size=IMG_SIZE,
                                               batch_size=BATCH_SIZE,
                                               class_mode='categorical',
                                               shuffle=True,
                                               subset = 'training') 

test_dataset = train_datagen.flow_from_directory(train_path, 
                                           target_size=IMG_SIZE,
                                           batch_size=BATCH_SIZE,
                                           class_mode='categorical',
                                           shuffle=False,
                                           subset = 'validation')

Found 70306 images belonging to 38 classes.
Found 17561 images belonging to 38 classes.


In [27]:
model = models.Sequential()

model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.BatchNormalization())

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.BatchNormalization())

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(train_dataset.num_classes, activation='sigmoid'))

model.summary()

/Users/sammamsohail/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_20 (Conv2D)              │ (None, 126, 126, 128)  │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 63, 63, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 63, 63, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 61, 61, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 61, 61, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 30, 30, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 12, 12, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 38)             │         4,902 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 525,734 (2.01 MB)

 Trainable params: 525,350 (2.00 MB)

 Non-trainable params: 384 (1.50 KB)

In [28]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [29]:
history = model.fit(
    train_dataset,
    epochs=4,
    verbose=1
)

Epoch 1/4
2198/2198 ━━━━━━━━━━━━━━━━━━━━ 282s 127ms/step - accuracy: 0.3914 - loss: 2.1509
Epoch 2/4
2198/2198 ━━━━━━━━━━━━━━━━━━━━ 1155s 525ms/step - accuracy: 0.7279 - loss: 0.8760
Epoch 3/4
2198/2198 ━━━━━━━━━━━━━━━━━━━━ 282s 128ms/step - accuracy: 0.8119 - loss: 0.5925
Epoch 4/4
2198/2198 ━━━━━━━━━━━━━━━━━━━━ 1089s 495ms/step - accuracy: 0.8448 - loss: 0.4986


In [34]:
model.save('model2.h5')

In [35]:
import numpy as np

predictions = model.predict(test_dataset)
predicted_classes = np.argmax(predictions, axis=1)

549/549 ━━━━━━━━━━━━━━━━━━━━ 31s 56ms/step


In [36]:
from sklearn.metrics import accuracy_score

true_labels = test_dataset.classes
accuracy = accuracy_score(true_labels, predicted_classes)
print('Accuracy:', accuracy)

Accuracy: 0.8718751779511418


In [37]:
from sklearn.metrics import classification_report

report = classification_report(true_labels, predicted_classes)
print(report)

              precision    recall  f1-score   support

           0       0.80      0.98      0.88       504
           1       0.94      0.90      0.92       496
           2       0.94      0.89      0.92       440
           3       0.88      0.90      0.89       502
           4       0.89      0.89      0.89       454
           5       0.91      0.97      0.94       420
           6       0.91      0.96      0.94       456
           7       0.86      0.79      0.83       410
           8       0.96      0.95      0.95       476
           9       0.92      0.88      0.90       477
          10       0.96      1.00      0.98       464
          11       0.55      0.99      0.71       472
          12       0.96      0.45      0.62       480
          13       0.81      0.89      0.85       430
          14       0.87      0.98      0.92       423
          15       0.87      0.98      0.92       502
          16       0.87      0.84      0.86       459
          17       0.96    